### Given a matrix: produce a matrix in row echelon form (REF).###

By using elementary row operations: interchange, scaling and replacement.<br>
**Definition:** A matrix is in REF if it satisfies the following conditions:<br>
-  If it has zero rows, it must be at the bottom of the matrix.<br>
-  Leading entry in each row non-zero row is 1. <br>
-  Each leadin is the to the right of all leading 1's above it.<br>


In [97]:
import numpy as np
import copy
def print_useful(g):
    'after one iteration, before update:' 
    print 'g.row',g.row,'g.col',g.col
    print 'g.leading',g.leading
    print g.m
    print 'done:',g.done()
    print '-------------------------------------'

class Gaussian:
    def __init__(self, matrix):
        self.m = np.array(self.float_it(matrix))
        self.col = 0
        self.row = 0
        self.leading = None
        self.num_zero_rows = self.number_zero_rows()
        self.N, self.M = self.m.shape
        self.pivot_pos = []
        self.isREF = False
    
    def all_zeros(self):
        return not np.any(self.m)
    def number_zero_rows(self):
        count = 0
        for row in self.m:
            if (not np.any(row)):
                count +=1
        return count
                    
    def float_it(self,matrix):
        for i,row in enumerate(matrix):
            for j,col in enumerate(row):
                matrix[i][j] = float(matrix[i][j])
        return matrix
    
    def interchange(self,row_i,row_j):
        tmp = self.m[row_j].copy()
        self.m[row_j] = self.m[row_i]
        self.m[row_i] = tmp
    
    def scale(self,K,row_i):
        scaled_row = K*self.m[row_i]
        self.m[row_i] = scaled_row
        
    def scale_for_rep(self,K,row_i):
        matrix = self.m.copy()
        return -K*matrix[row_i]
    
    def replace(self,K,row_i,row_j):
        scaled_row = self.scale_for_rep(K,row_i)
        replace_row = self.m[row_j]
        self.m[row_j] = [i + j for i,j in zip(scaled_row, replace_row)]
        if (not np.any(self.m[row_j])):
            self.num_zero_rows +=1
        
    def update(self,r,c,l):
        self.row = r
        self.col = c
        self.pivot_pos.append((r,c))
        self.leading = l
        
    def reset_leading(self):
        self.leading = None
        self.row +=1
        self.col +=1
    
    def find_one(self):
        for r_idx,elt in enumerate (self.m.T[self.col]):
            if (elt == 1):
                self.interchange(r_idx,self.row)
                self.leading = 1
                break
        if (self.leading != 1 and self.leading != None):
            K = 1/float(self.leading)
            self.scale(K,self.row)
    
    
    def do_replacements(self):
        for r_idx,elt in enumerate(self.m.T[self.col]):
            if (elt != 0 and r_idx > self.row):
                K = elt
                self.replace(K,self.row,r_idx)
                
    def zero_to_bot(self,r_idx):
        while (r_idx > self.row):
            new_r_idx = r_idx - 1
            self.interchange(r_idx,new_r_idx)
            r_idx -= 1
        return r_idx
    
    def first_none_zero_elt(self,col):
        for r_idx in range(self.row,self.N):
            elt = col[r_idx]
            if (elt != 0):
                return r_idx,elt
        return -1,-1
        
    def find_leading(self):
        matrixT = self.m.T
        for c_idx in range(self.col,self.M):
            col = self.m.T[c_idx]
            not_all_zero = np.any(col)
            if ( (not_all_zero)):
                r_idx,elt = self.first_none_zero_elt(col)
                if (r_idx == -1):
                    continue
                else:
                    r_idx = self.zero_to_bot(r_idx)
                    self.update(r_idx,c_idx,elt)
                    break
        if (self.leading !=1):
            self.find_one()
        
    #are we done yet? 
    def done(self):
        all_rows_seen = self.row + self.num_zero_rows >= self.N 
        all_cols_seen = self.col  >=  self.M 
        return all_rows_seen and all_cols_seen
    
    def produce_REF(self):
        while(not self.done()):
            self.find_leading()
            self.do_replacements()
            if (self.leading == None):
                break
            self.reset_leading()
        return self.m
    #--------RREF--------------
   
    def produce_RREF(self):
        pps = self.pivot_pos[::-1]
        matrixT = self.m.T
        for pp in pps:
            ri,ci = pp
            o_ri = ri
            col = matrixT[ci]
            leading = col[o_ri]
            while (ri > 0):
                ri = ri - 1
                K = col[ri]
                if (K!= 0):
                    self.replace(K,o_ri,ri)
        return self.m

In [98]:
def solve(matrix):
    g = Gaussian(matrix)
    return g.produce_REF()

### Given a matrix: produce a matrix in reduced row echelon form (RREF).###

By using elementary row operations: interchange, scaling and replacement.<br>
**Definition:** A matrix is in RREF if it satisfies the following conditions:<br>
-  The matrix is in REF 
-  Each leading 1 is the only non zero number in it's column

In [99]:
def solve_RREF(matrix):
    g = Gaussian(matrix)
    g.produce_REF()
    g.produce_RREF()
    return g.m
    